In [0]:
from pyspark.sql.types import StructType
from pyspark.sql.functions import current_timestamp, from_utc_timestamp
import delta


In [0]:
path_schema = '/Volumes/raw/tabnews/datalake/Ano_2022/Mês_05/Dia_06/Minuto_15/Segundo_20/*.json'
sample_df = spark.read.option('multiline', 'true').json(path_schema)

In [0]:
path = '/Volumes/raw/tabnews/datalake/Ano_*/Mês_*/Dia_*/Minuto_*/Segundo_*/*.json'
checkpoint_path = '/Volumes/raw/tabnews/datalake/checkpoint'
schema = sample_df.schema

df_streaming = (spark.readStream.format("cloudFiles")
                                .option("cloudFiles.format", "json")
                                .option("cloudFiles.schemaLocation", checkpoint_path)
                                .option("multiLine", "true")
                                .schema(schema)
                                .load(path))

In [0]:
delta_table = delta.DeltaTable.forName(spark, 'bronze.tabnews.Bronze_TabNews_Stream')
query = """
SELECT *, 
       ROW_NUMBER() OVER (PARTITION BY id ORDER BY UPDATED_AT DESC) as row_num 
FROM global_temp.table_temp"""

def upsert(df, delta_table, query):
    df.createOrReplaceGlobalTempView('table_temp')
    df_transformacao = spark.sql(query).filter("row_num = 1").drop("row_num")
    (delta_table.alias("d")
                .merge(df_transformacao.alias("n"), 'd.id = n.id')
                .whenMatchedUpdateAll()
                .whenNotMatchedInsertAll()
                .execute())


stream = (df_streaming.writeStream
                      .foreachBatch(lambda df, batchID: upsert(df, delta_table, query))
                      .option("checkpointLocation", checkpoint_path)
                      .trigger(availableNow=True)
                      .start())

In [0]:
path = '/Volumes/raw/tabnews/datalake/Ano_2022/Mês_05/Dia_06/Minuto_15/Segundo_20/*.json'
df = (spark.read.option('multiline', 'true')
                .schema(sample_df.schema)
                .json(path))
df.createOrReplaceTempView('Raw_TabNews_LoadFull')

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW somente_ultima_atualizacao AS
SELECT * 
FROM Raw_TabNews_LoadFull
QUALIFY ROW_NUMBER() OVER (PARTITION BY id ORDER BY UPDATED_AT DESC) = 1

In [0]:
df_somente_ultima_atualizacao = spark.table("somente_ultima_atualizacao")

df_somente_ultima_atualizacao.write.mode('overwrite') \
                              .format('delta') \
                              .option('overwriteSchema', 'true') \
                              .option('mergeSchema', 'true') \
                              .saveAsTable('bronze.tabnews.Bronze_TabNews_Stream')